**Time-Series Visualization using Bokeh**

### Import Libraries

In [6]:
import numpy as np, pandas as pd
import warnings
warnings.filterwarnings('ignore')
from bokeh.plotting import figure, output_file, show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
    FactorRange
)
import bokeh.palettes
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook

In [7]:
output_notebook()

Loading BokehJS ...

### Loading Data

In [8]:
data = pd.read_csv("../datasets/Monthly_Property_Crime_2005_to_2015.csv")

In [9]:
data.shape

(792, 3)

In [10]:
data.head(5)

,Date,Category,IncidntNum
0,02/01/2014 12:00:00 AM,BURGLARY,506
1,02/01/2007 12:00:00 AM,VANDALISM,531
2,07/01/2012 12:00:00 AM,BURGLARY,522
3,07/01/2013 12:00:00 AM,LARCENY/THEFT,3318
4,08/01/2010 12:00:00 AM,VANDALISM,694


### Extracting Year, Month from Date column

In [11]:
data["Date"] = data["Date"].astype("category").str.slice(0,10)
data["Date"] = pd.to_datetime(data["Date"])

In [12]:
data["Year"] = data["Date"].dt.year
data["Month"] = data["Date"].dt.month

In [13]:
data.head(3)

,Date,Category,IncidntNum,Year,Month
0,2014-02-01,BURGLARY,506,2014,2
1,2007-02-01,VANDALISM,531,2007,2
2,2012-07-01,BURGLARY,522,2012,7


### 1.  Bar Chart

Grouping data by years to see the trend of criminal incidents vs year

In [14]:
total_by_year = data.groupby(["Year"]).IncidntNum.sum().reset_index()

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title = "Total Number of Criminal Incidents vs Year", 
           x_axis_label = 'Year', 
           y_axis_label = 'Number of Criminal Incidents',
           tools=TOOLS)

p.vbar(x=total_by_year["Year"],
       top=total_by_year["IncidntNum"], 
       width = 1,
       line_color="white" )

show(p)

### 2. HeatMap

In [15]:
total_by_month = data.groupby(["Year","Month"]).IncidntNum.sum().reset_index()

In [16]:
newdata = ColumnDataSource(total_by_month)

In [17]:
mapper = LinearColorMapper(palette=bokeh.palettes.RdBu[9], 
                           low=total_by_month.IncidntNum.min(), high=total_by_month.IncidntNum.max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title = "Total Number of Criminal Incidents",
           x_axis_label = 'Year', 
           y_axis_label = 'Month',
           tools=TOOLS,
           plot_width = 600)

p.rect(x="Year", y="Month",width=1, height=1,source = newdata,
       fill_color={'field': 'IncidntNum', 'transform': mapper})

color_bar = ColorBar(color_mapper=mapper, location=(18, 0),
                     ticker=BasicTicker(desired_num_ticks=9))

p.add_layout(color_bar, 'right')
show(p)

### 3. Multi-Line Plot

In [18]:
grouped_data = data.groupby(["Year","Category"]).IncidntNum.sum().reset_index()

In [19]:
dict_group_data = {"xs":[grouped_data.loc[(grouped_data["Category"]==name),"Year"].values \
                         for name in grouped_data['Category'].unique()],
                  "ys": [grouped_data.loc[(grouped_data["Category"]==name),"IncidntNum"].values\
                         for name in grouped_data['Category'].unique()],
                  "labels": grouped_data["Category"].unique(),
                  "color": bokeh.palettes.Category10[6]}

source = ColumnDataSource(dict_group_data)

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(x_axis_label = 'Year', 
           y_axis_label = 'Number of Incidents', 
           width = 700,
           title = "Number of Yearly Criminal Incidents by each Category",
           tools=TOOLS) 

p.multi_line(xs= "xs", ys= "ys", line_color= "color", source = source, legend = "labels",line_width=3)
p.legend.location = "top_left"

show(p)

### 4. Stacked Bar Charts

In [20]:
categories = grouped_data["Category"].unique()
years = ['2013','2014', '2015']

data_2 = {'Categories' : categories,
        '2013'   : grouped_data.loc[(grouped_data["Year"]==2013),"IncidntNum"],
        '2014'   : grouped_data.loc[(grouped_data["Year"]==2014),"IncidntNum"],
        '2015'   : grouped_data.loc[(grouped_data["Year"]==2015),"IncidntNum"]}


x = [ (category, year) for category in categories for year in years ]
counts = sum(zip(data_2['2013'], data_2['2014'], data_2['2015']),()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(x_range=FactorRange(*x), 
           height=500,
           width = 700,
           title="Incidents by Category for last 3 Years",
           toolbar_location=None, 
           tools=TOOLS, 
           y_axis_label = 'Number of Incidents')

p.vbar(x='x', 
       top='counts', 
       width=0.9, 
       source=source, 
       line_color="white",
       fill_color=factor_cmap('x', palette=bokeh.palettes.Category10[3], factors=years, start=1, end=2))

show(p)